Simple 1D bar problem
-------------------------------

Let's define a static equilibrium of 2 bars defined by the following matrices:

* Bar 1
$$
\mathbf{K_1} =
\begin{bmatrix}
2 & -1 \\
-1 & 1
\end{bmatrix}
$$

$$
\mathbf{B_1} =
\begin{bmatrix}
0 & 1
0 & 1
0 & 1
\end{bmatrix}
$$

$$
\mathbf{f_1} =
\begin{bmatrix}
0  \\
0
\end{bmatrix}
$$

* Bar 2
$$
\mathbf{K_2} =
\begin{bmatrix}
 1 & -1 \\
-1 & 2
\end{bmatrix}
$$

$$
\mathbf{B_2} =
\begin{bmatrix}
-1 & 0
-1 & 0
-1 & 0
\end{bmatrix}
$$



$$
\mathbf{f_2} =
\begin{bmatrix}
0  \\
1
\end{bmatrix}
$$

These matrices discrebe two bars, where Bar 1 is fixed in the left and the Bar 2 fixed in the right.

The equilibrium of these two bars are given by:

$$
\mathbf{K_1} \mathbf{u_1} = \mathbf{f_1} \\
\mathbf{K_2}  \mathbf{u_2} = \mathbf{f_2} \\
$$

Now, we introduce the compatibility contraints between these two bars, such that:


$$
\mathbf{B_1} \mathbf{u_1} + \mathbf{B_2} \mathbf{u_2} = \mathbf{0} \\
$$

Then, we can write the hybrid problem as:

$$
\begin{bmatrix}
\mathbf{K} &  \mathbf{B^T}  \\
\mathbf{B} & \mathbf{0}
\end{bmatrix}
\begin{bmatrix}
\mathbf{u}  \\
\mathbf{\lambda}
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{f}  \\
\mathbf{0}
\end{bmatrix}
$$

In [1]:
import numpy as np
from scipy.sparse import linalg
from scipy.sparse import coo_matrix, bmat, hstack, vstack, linalg
from pyfeti.src.feti_solver import SerialFETIsolver


# A simple example with Reduntant Constraints Positive Define Domains
K1 = np.array([[2,-1],[-1,1]])
K2 = np.array([[1,-1],[-1,2]])
B1 = np.array([[0,1],[0,1],[0,1]]) 
B2 = np.array([[-1,0],[-1,0],[-1,0]]) 
f1 = np.array([0.,0.])                
f2 = np.array([0.,1.])                
               

    
K = bmat([[K1,None],[None,K2]])
B = np.array(np.bmat([[B1,B2]]))
nc, ndof = B.shape
f = np.concatenate([f1,f2])
b = np.concatenate([f,np.zeros(nc)])
A = bmat([[K,B.T],[B,None]]).A

In [2]:
A

array([[ 2, -1,  0,  0,  0,  0,  0],
       [-1,  1,  0,  0,  1,  1,  1],
       [ 0,  0,  1, -1, -1, -1, -1],
       [ 0,  0, -1,  2,  0,  0,  0],
       [ 0,  1, -1,  0,  0,  0,  0],
       [ 0,  1, -1,  0,  0,  0,  0],
       [ 0,  1, -1,  0,  0,  0,  0]], dtype=int32)

In [3]:
x, info = linalg.cg(A,b)
u = x[:ndof]
lambda_ = x[-nc:]

In [4]:
A.dot(x)  - b

array([ 5.55111512e-17,  1.80411242e-16, -1.38777878e-16, -1.11022302e-16,
       -1.66533454e-16, -1.66533454e-16, -1.66533454e-16])

In [5]:
# dual formulation
K_inv = np.linalg.inv(K.A)
F = B@K_inv@B.T
d = np.array(B@K_inv@f).flatten()

In [6]:
x_cg, info = linalg.cg(F,d)

In [7]:
F@x_cg - d

array([0., 0., 0.])

In [8]:
x_cg - lambda_

array([-8.32667268e-17, -8.32667268e-17, -8.32667268e-17])

In [9]:
eigval, eigvec = np.linalg.eig(F)

eigval

array([ 1.20000000e+01,  2.46519033e-32, -3.00385456e-16])

In [10]:
eigvec

array([[ 5.77350269e-01, -6.51147040e-17,  6.09781659e-01],
       [ 5.77350269e-01, -7.07106781e-01, -7.75129861e-01],
       [ 5.77350269e-01,  7.07106781e-01,  1.65348202e-01]])

In [11]:
x0 = np.array(eigvec[:,1]).flatten()
x_cg, info = linalg.cg(F,d,x0)

In [12]:
info

0

In [13]:
x_cg

array([-0.08333333, -0.79044011,  0.62377345])

In [14]:
 x_cg - lambda_

array([-2.22044605e-16, -7.07106781e-01,  7.07106781e-01])

In [15]:
F@x_cg - d

array([4.4408921e-16, 4.4408921e-16, 4.4408921e-16])

In [16]:
u_cg = K_inv@(f - B.T@x_cg)

In [17]:
K_inv@(f - B.T@x_cg)

array([0.25, 0.5 , 0.5 , 0.75])

In [18]:
B

array([[ 0,  1, -1,  0],
       [ 0,  1, -1,  0],
       [ 0,  1, -1,  0]])

In [19]:
f

array([0., 0., 0., 1.])

In [20]:
u_cg

array([0.25, 0.5 , 0.5 , 0.75])

In [21]:
u

array([0.25, 0.5 , 0.5 , 0.75])

In [22]:
# Using PyFETI to solve the probrem described above
K_dict = {1:K1,2:K2}
B_dict = {1 : {(1,2) : B1}, 2 : {(2,1) : B2}}
f_dict = {1:f1,2:f2}

solver_obj = SerialFETIsolver(K_dict,B_dict,f_dict)

solution_obj = solver_obj.solve()

u_dual = solution_obj.displacement
lambda_ = solution_obj.interface_lambda
alpha =  solution_obj.alpha


In [23]:
u_dual

array([0.25, 0.5 , 0.5 , 0.75])